In [1]:
import polars as pl
import os
import numpy as np
import torch, torchvision
from torchvision.utils import make_grid
from glio.loaders import nifti
from glio.python_tools import get_all_files
from glio.jupyter_tools import clean_mem
from glio.visualize import Visualizer
import matplotlib.pyplot as plt
path = r"E:\dataset\UCSF-PDGM-v3"

In [2]:
all_files = get_all_files(f"{path}/UCSF-PDGM-v3", extensions=["nii.gz"])
id_length = len("UCSF-PDGM-0004")
files_dict = {}

for file in all_files:
    filename = os.path.basename(file)
    study_id = filename[:id_length]
    file_modality = f"PATH{filename[id_length:]}"
    if study_id not in files_dict:
        files_dict[study_id] = {"ID": study_id}
    files_dict[study_id][file_modality] = file

files_dict = list(files_dict.values())

filepaths_df = pl.from_dicts(files_dict)
filepaths_df

ID,PATH_ADC.nii.gz,PATH_ASL.nii.gz,PATH_brain_parenchyma_segmentation.nii.gz,PATH_brain_segmentation.nii.gz,PATH_DTI_eddy_FA.nii.gz,PATH_DTI_eddy_L1.nii.gz,PATH_DTI_eddy_L2.nii.gz,PATH_DTI_eddy_L3.nii.gz,PATH_DTI_eddy_MD.nii.gz,PATH_DTI_eddy_noreg.nii.gz,PATH_DWI.nii.gz,PATH_DWI_bias.nii.gz,PATH_FLAIR.nii.gz,PATH_FLAIR_bias.nii.gz,PATH_SWI.nii.gz,PATH_SWI_bias.nii.gz,PATH_T1.nii.gz,PATH_T1c.nii.gz,PATH_T1c_bias.nii.gz,PATH_T1_bias.nii.gz,PATH_T2.nii.gz,PATH_T2_bias.nii.gz,PATH_tumor_segmentation.nii.gz
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""UCSF-PDGM-0004…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…"
"""UCSF-PDGM-0005…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…"
"""UCSF-PDGM-0007…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…"
"""UCSF-PDGM-0008…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…"
"""UCSF-PDGM-0009…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""UCSF-PDGM-0537…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…"
"""UCSF-PDGM-0538…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\dataset\UCS…","""E:\da

In [3]:
filepaths_df.columns

['ID',
 'PATH_ADC.nii.gz',
 'PATH_ASL.nii.gz',
 'PATH_brain_parenchyma_segmentation.nii.gz',
 'PATH_brain_segmentation.nii.gz',
 'PATH_DTI_eddy_FA.nii.gz',
 'PATH_DTI_eddy_L1.nii.gz',
 'PATH_DTI_eddy_L2.nii.gz',
 'PATH_DTI_eddy_L3.nii.gz',
 'PATH_DTI_eddy_MD.nii.gz',
 'PATH_DTI_eddy_noreg.nii.gz',
 'PATH_DWI.nii.gz',
 'PATH_DWI_bias.nii.gz',
 'PATH_FLAIR.nii.gz',
 'PATH_FLAIR_bias.nii.gz',
 'PATH_SWI.nii.gz',
 'PATH_SWI_bias.nii.gz',
 'PATH_T1.nii.gz',
 'PATH_T1c.nii.gz',
 'PATH_T1c_bias.nii.gz',
 'PATH_T1_bias.nii.gz',
 'PATH_T2.nii.gz',
 'PATH_T2_bias.nii.gz',
 'PATH_tumor_segmentation.nii.gz']